# Run original and READII negative control CTs through FMCIB

This notebook is set up to use outputs from the `run_readii_prep.ipynb` notebook. 

CT images cropped around a Gross Tumour Volume (GTV), resized, and saved as nifti files are fed into an instance of FMCIB for inference and all features are saved.

## Set up pixi environment kernel

1. Run the following commands in the terminal:

    ```bash
    $ pixi install

    $ pixi shell -e fmcib

    $ ipython kernel install --user --name=fmcib --display-name=fmcib
    ```

2. In the `Select Kernel` menu at the top right of the notebook, select `Jupyter Kernel` as the source. 

3. Refresh the options and one called `fmcib` should appear. Select this option.

## Imports

In [1]:
import yaml
import itertools
from pathlib import Path

import sys; sys.path.append("code")
from infer import runFMCIBInfer

## Initialize dataset name and negative control settings

In [2]:
config = yaml.load(open("config/RADCURE.yaml", "r"), Loader=yaml.FullLoader)

DATASET_NAME = config["dataset_name"]
NEG_CONTROL_REGIONS = config["negative_control_regions"]
NEG_CONTROL_TYPES = config["negative_control_types"]

## Set up data directories

In [3]:
# Set up results directory
features_dir = Path("results", DATASET_NAME, "fmcib_features")
Path.mkdir(features_dir, parents=True, exist_ok=True)

correlations_dir = Path("results", DATASET_NAME, "analysis", "correlations")
Path.mkdir(correlations_dir, parents=True, exist_ok=True)

## Generate list of image types to run through FMCIB

In [4]:
# Get full list of image types to run FMCIB on
negative_controls = [f"{negative_control[0]}_{negative_control[1]}" for negative_control in itertools.product(NEG_CONTROL_TYPES, NEG_CONTROL_REGIONS)]
image_types = ["original"] + negative_controls

## Run each image type through FMCIB and save out the resulting features

In [6]:
for image_type in image_types:
    # Find and load in the FMCIB input csv file
    fmcib_input_csv_path = Path("procdata", DATASET_NAME, "images", "fmcib_input", f"fmcib_input_{DATASET_NAME}_{image_type}.csv")

    feature_file_save_path = Path(features_dir, f"fmcib_features_{DATASET_NAME}_{image_type}.csv")

    runFMCIBInfer(feature_file_save_path=str(feature_file_save_path), 
                  csv_path=str(fmcib_input_csv_path), 
                  weights_path=None,
                  precropped=True)

2024-12-17 18:39:48.940 | INFO     | fmcib.run:get_features:25 - Loading CSV file ...
2024-12-17 18:39:48.944 | INFO     | fmcib.preprocessing:get_dataloader:48 - Building dataloader instance ...
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f